In [1]:
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras import optimizers
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.layers import BatchNormalization
from sklearn.metrics import accuracy_score
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint


/home/rithwik/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = (X_train - 127.5) / 127.5
X_test = (X_test - 127.5) / 127.5

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [4]:
seed= 100
np.random.seed(seed)

In [15]:
 
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [6]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003       
    return lrate

In [12]:
batch_size = 64


opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size,epochs=125,\
                    verbose=1,validation_data=(X_test,y_test),callbacks=[LearningRateScheduler(lr_schedule),checkpoint])
#save to disk









Train on 50000 samples, validate on 10000 samples
Epoch 1/125
50000/50000 [==============================] - 123s 2ms/step - loss: 1.6404 - acc: 0.4637 - val_loss: 1.1433 - val_acc: 0.6050

Epoch 00001: val_acc improved from -inf to 0.60500, saving model to weights-improvement-01-0.60.hdf5
Epoch 2/125
50000/50000 [==============================] - 126s 3ms/step - loss: 1.0467 - acc: 0.6532 - val_loss: 0.9932 - val_acc: 0.6789

Epoch 00002: val_acc improved from 0.60500 to 0.67890, saving model to weights-improvement-02-0.68.hdf5
Epoch 3/125
50000/50000 [==============================] - 119s 2ms/step - loss: 0.8862 - acc: 0.7171 - val_loss: 0.8256 - val_acc: 0.7412

Epoch 00003: val_acc improved from 0.67890 to 0.74120, saving model to weights-improvement-03-0.74.hdf5
Epoch 4/125
50000/50000 [==============================] - 121s 2ms/step - loss: 0.7985 - acc: 0.7520 - val_loss: 0.7757 - val_acc: 0.7694

Epoch 00004: val_acc improved from 0.74120 to 0.76940, saving model to weights-im

KeyboardInterrupt: 

In [18]:
model.load_weights("/home/rithwik/Desktop/inter iit/Star Cluster/weights-improvement-33-0.85.hdf5")

In [26]:
from sklearn.metrics import confusion_matrix, classification_report

y_hat = model.predict(X_test)
y_pred = [np.argmax(y_hat[i]) for i in range(len(y_hat))]
y_true = [np.argmax(y_test[i]) for i in range(len(y_test))]

print(confusion_matrix(y_true, y_pred))




[[883  14  14   5  15   1   3  10  35  20]
 [  7 944   0   2   2   1   0   0   7  37]
 [ 53   5 744  26  84  20  28  28   6   6]
 [ 21   6  44 621  66 126  39  52  11  14]
 [  7   4  15   9 890  12   6  51   6   0]
 [ 12   7  30  87  37 749  12  57   1   8]
 [  8   5  23  28  42   9 873   7   3   2]
 [  6   1   5  11  22  13   2 938   0   2]
 [ 34  21   3   2   3   1   3   4 914  15]
 [ 19  56   1   4   1   0   1   5  14 899]]


In [27]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1000
           1       0.89      0.94      0.92      1000
           2       0.85      0.74      0.79      1000
           3       0.78      0.62      0.69      1000
           4       0.77      0.89      0.82      1000
           5       0.80      0.75      0.78      1000
           6       0.90      0.87      0.89      1000
           7       0.81      0.94      0.87      1000
           8       0.92      0.91      0.92      1000
           9       0.90      0.90      0.90      1000

   micro avg       0.85      0.85      0.85     10000
   macro avg       0.85      0.85      0.84     10000
weighted avg       0.85      0.85      0.84     10000



# NN + SVM

In [ ]:
model.layers.pop(-1)

In [29]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [30]:
from sklearn.svm import SVC

In [31]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [32]:
classifier = SVC()
classifier.fit(train_pred,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [33]:
test_pred_labels = classifier.predict(test_pred)

print(accuracy_score(y_test,test_pred_labels))

0.8509


In [37]:
print(confusion_matrix(y_test, test_pred_labels))

[[879  10  24  11  13   1   2   6  35  19]
 [  6 938   0   3   2   1   0   0   8  42]
 [ 42   4 778  33  60  21  31  20   6   5]
 [ 14   4  47 690  40 119  35  32   8  11]
 [ 10   4  24  26 864  20   9  38   5   0]
 [ 10   7  29 110  30 759  12  36   0   7]
 [  8   5  27  43  28   8 873   3   3   2]
 [  5   1  10  20  27  26   1 908   0   2]
 [ 32  15   3   5   3   1   3   3 916  19]
 [ 20  49   2   5   1   0   1   4  14 904]]
